In [1]:
import os

In [2]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "${PROJECT}

Your current GCP Project Name is: qwiklabs-gcp-04-014eea1283f0


In [3]:
# TODO: Change these to try this notebook out
PROJECT = "qwiklabs-gcp-04-014eea1283f0"  # Replace with your PROJECT
BUCKET = "chest-xray-us-central"   # defaults to PROJECT
REGION = "us-central1"  # Replace with your REGION

In [4]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.1"
os.environ["PYTHONVERSION"] = "3.7"

In [5]:
%%bash
gcloud config set project ${PROJECT}
gcloud config set compute/region ${REGION}

Updated property [core/project].
Updated property [compute/region].


In [6]:
%%bash
gsutil ls gs://${BUCKET}/chest_xray/

gs://chest-xray-us-central/chest_xray/chest_xray_labels.csv
gs://chest-xray-us-central/chest_xray/test/
gs://chest-xray-us-central/chest_xray/train/
gs://chest-xray-us-central/chest_xray/val/


In [7]:
import os
import shutil
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib

import cv2

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dense, Flatten, Softmax

import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

In [8]:

CLASS_LABELS = ['NORMAL', 'PNEUMONIA'] 

def process_path(nb_class):
    
    def f(file_path):
        
        label = 0 if tf.strings.split(file_path, os.path.sep)[-2]=='NORMAL' else 1
            
        image = tf.io.read_file(file_path)    
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
         
        image = tf.image.resize(image, [127, 127], method='area')
        return image, label
    
    return f

def reader_image(path_file, batch_size, nb_class):

    list_ds = tf.data.Dataset.list_files(path_file)
    labeled_ds = list_ds.map(process_path(nb_class))
    
    return labeled_ds.shuffle(100).batch(batch_size).prefetch(1)

train_ds = reader_image('gs://chest-xray-us-central/chest_xray/train/*/*.jpeg', 16, 2)
test_ds = reader_image('gs://chest-xray-us-central/chest_xray/test/*/*.jpeg', 16, 2)
test1_ds = reader_image('gs://chest-xray-us-central/chest_xray/val/*/*.jpeg', 16, 2)
print(type(train_ds))

# We display an example
for image, label in train_ds.take(1):
    df = pd.DataFrame(image[0, :, :, 0].numpy())
    
print(f'Outoupt : \n image shape: {df.shape}')

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
Outoupt : 
 image shape: (127, 127)


In [9]:
from sklearn.model_selection import train_test_split

def get_numpy(ds):
    

    X, y = [], []
    for images, labels in ds: 
        X.append(images)
        y.append(labels)
        
    X = np.concatenate(X, axis=0)
    X = X.reshape(X.shape[0], -1)
    y = np.concatenate(y, axis=0)
    
    return X, y

X, y = get_numpy(train_ds)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)
X_test, y_test = get_numpy(test_ds)

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(20, 10))

for images, labels in train_ds.take(1):
    for i in range(6):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i, :, :, 0].numpy())
        plt.title(f'Label: {CLASS_LABELS[labels[i].numpy()]}')
        plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

base = DenseNet169(weights = 'imagenet', include_top = False, input_shape = (127, 127, 3))
tf.keras.backend.clear_session()

for layer in base.layers:
    layer.trainable =  False # freezing densenet layers 

densenet_model = Sequential()
densenet_model.add(base)
densenet_model.add(GlobalAveragePooling2D())
densenet_model.add(BatchNormalization())
densenet_model.add(Dense(256, activation='relu'))
densenet_model.add(Dropout(0.5))
densenet_model.add(BatchNormalization())
densenet_model.add(Dense(128, activation='relu'))
densenet_model.add(Dropout(0.5))
densenet_model.add(Dense(1, activation='sigmoid'))

densenet_model.summary()

In [ ]:
optm = Adam(lr=0.0001)
densenet_model.compile(loss='binary_crossentropy', optimizer=optm, 
                  metrics=['accuracy'])

In [ ]:
dense_history = densenet_model.fit(
              train_ds,
              validation_data=test_ds,
              epochs=2)

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

base = MobileNetV2(weights = 'imagenet', include_top = False, input_shape= (127, 127, 3))
tf.keras.backend.clear_session()
    
for layer in base.layers:
    layer.trainable =  False

mobilenet_model = Sequential()
mobilenet_model.add(base)
mobilenet_model.add(GlobalAveragePooling2D())
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(Dense(256, activation='relu'))
mobilenet_model.add(Dropout(0.5))
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(Dense(128, activation='relu'))
mobilenet_model.add(Dropout(0.5))
mobilenet_model.add(Dense(1, activation='sigmoid'))

mobilenet_model.summary()

In [ ]:
optm = Adam(lr=0.0001)
mobilenet_model.compile(loss='binary_crossentropy', optimizer=optm, 
                  metrics=['accuracy'])

In [ ]:
mobilenet_history = mobilenet_model.fit(
              train_ds,
              validation_data=test_ds,
              epochs=20)

In [ ]:
from keras.layers.merge import concatenate
from keras.layers import Input
import tensorflow as tf

input_shape = (224,224,3)
input_layer = Input(shape = (224, 224, 3))

#first model
base_mobilenet = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = input_shape)
base_densenet = DenseNet169(weights = 'imagenet', include_top = False, input_shape = input_shape)

for layer in base_mobilenet.layers:
    layer.trainable =  False
for layer in base_densenet.layers:
    layer.trainable = False
    
model_mobilenet = base_mobilenet(input_layer)
model_mobilenet = GlobalAveragePooling2D()(model_mobilenet)
output_mobilenet = Flatten()(model_mobilenet)

model_densenet = base_densenet(input_layer)
model_densenet = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet)

merged = tf.keras.layers.Concatenate()([output_mobilenet, output_densenet]) 

x = BatchNormalization()(merged)
x = Dense(256,activation = 'relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = 'sigmoid')(x)
stacked_model = tf.keras.models.Model(inputs = input_layer, outputs = x)
stacked_model.summary()

In [ ]:
optm = Adam(lr=0.0001)
stacked_model.compile(loss='binary_crossentropy', optimizer=optm, 
                  metrics=['accuracy'])

In [ ]:
stacked_history = stacked_model.fit( 
                train_ds,
              validation_data=test_ds,
              epochs=20)